In [ ]:
# Run this code to analyse the whole database of videos and process it

### 1)Imports and setup

In [14]:
import os
import base64
import aiohttp
import asyncio
import json
import imageio
import re
import time
from PIL import Image
import numpy as np
import colorsys
import aiofiles
import nest_asyncio
from tqdm.asyncio import tqdm
from dotenv import load_dotenv
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
import logging
from datetime import datetime
import csv
from rapidfuzz import process, fuzz


# Configure logging
logging.basicConfig(level=logging.INFO)

# Load OpenAI API key from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Apply nest_asyncio to handle the running event loop
nest_asyncio.apply()

# Concurrency limit
semaphore = asyncio.Semaphore(5)

# A dictionary to store characters across frames
character_frames = {}


In [15]:
#TRACK API USAGE CALLS

# Initialize API usage tracking
api_usage = {
    "total_api_calls": 0,
    "total_tokens_used": 0,
    "model_used": "gpt-4"  # Assuming you're using GPT-4
}


### 2) Video Analysis Functions

In [16]:
def analyze_video(video_path, threshold=27.0):
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"The video file {video_path} does not exist.")
    
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=threshold))

    video_manager.set_downscale_factor()
    video_manager.start()

    scene_manager.detect_scenes(frame_source=video_manager)
    scene_list = scene_manager.get_scene_list()

    video_manager.release()

    logging.info(f'Detected {len(scene_list)} scenes:')
    for i, scene in enumerate(scene_list):
        logging.info(f'Scene {i + 1}: Start {scene[0].get_timecode()} / Frame {scene[0].get_frames()}, '
              f'End {scene[1].get_timecode()} / Frame {scene[1].get_frames()}')

    return scene_list

def get_video_length(video_path):
    # You can use a tool like OpenCV, ffmpeg, or similar to calculate video length
    import cv2
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_length = frame_count / fps
    cap.release()
    return video_length

### 3) Frame Extraction Function

In [17]:
def extract_frames_imageio(video_path, scenes, output_dir):
    reader = imageio.get_reader(video_path)
    for i, scene in enumerate(scenes):
        start_frame, end_frame = scene
        
        # Convert FrameTimecode to integer frame numbers
        start_frame_num = int(start_frame)
        end_frame_num = int(end_frame)
        
        # Calculate the middle frame of the scene
        middle_frame = (start_frame_num + end_frame_num) // 2
        
        # Set the reader to the middle frame and extract it
        reader.set_image_index(middle_frame)
        frame = reader.get_next_data()
        
        # Save the frame as an image with frame number in the filename
        output_path = os.path.join(output_dir, f'scene_{i + 1}_frame_{middle_frame}.jpg')
        imageio.imwrite(output_path, frame)
        print(f"Extracted and saved middle frame of scene {i + 1} as {output_path}", flush=True)


### 4) Image Processing Function

In [18]:
async def encode_image(image_path):
    async with aiofiles.open(image_path, "rb") as image_file:
        content = await image_file.read()
        return base64.b64encode(content).decode('utf-8')

def get_color_category(color):
    r, g, b = [x / 255.0 for x in color]
    h, l, s = colorsys.rgb_to_hls(r, g, b)

    primary_hues = {
        "red": (0.0, 0.1),  
        "yellow": (0.1, 0.18),
        "green": (0.25, 0.4),
        "blue": (0.55, 0.75),
    }

    for color_name, hue_range in primary_hues.items():
        if hue_range[0] <= h <= hue_range[1]:
            return color_name

    if (l >= 0.9 and s <= 0.1):
        return "white"
    if (l <= 0.1 and s <= 0.1):
        return "black"

    return "non-primary"

def analyze_image_colors(image_path):
    image = Image.open(image_path)
    image = image.convert('RGB')
    data = np.array(image)

    unique_colors, counts = np.unique(data.reshape(-1, data.shape[2]), axis=0, return_counts=True)
    total_pixels = int(counts.sum())

    color_counts = {
        "Red": 0,
        "Yellow": 0,
        "Green": 0,
        "Blue": 0,
        "White": 0,
        "Black": 0,
        "Non-primary": 0
    }

    for color, count in zip(unique_colors, counts):
        category = get_color_category(tuple(color))
        color_counts[category.capitalize()] += int(count)

    color_percentages = {color: (count / total_pixels) * 100 for color, count in color_counts.items()}
    primary_total = color_counts["Red"] + color_counts["Yellow"] + color_counts["Blue"]
    color_dominance = "Primary colors" if primary_total > color_counts["Non-primary"] else "Non-primary colors"

    return {
        "Color Analysis": {
            "Colors Found": {
                color: {
                    "Pixel Count": count,
                    "Percentage": f"{color_percentages[color]:.2f}%"
                } for color, count in color_counts.items()
            },
            "Dominance": color_dominance
        }
    }


### 5) OpenAI API Interaction

In [19]:
async def send_image_to_openai(image_path, base64_image, retries=3):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """
                        Analyze the following image and provide a detailed description in the format of JSON only. Ensure the output is strictly in JSON format without any additional text or code block formatting. The JSON should include the following standardized labels:

                        1. **Image Analysis**: The root dictionary containing all analysis data.
                        
                        2. **Suitability**:
                            - "Nudity": Boolean indicating the presence of nudity.
                            - "Obscene Gestures": Boolean indicating the presence of obscene gestures.
                            - "Alcohol": Boolean indicating the presence of alcohol.
                            - "Drugs": Boolean indicating the presence of drugs.
                            - "Addictions": Boolean indicating the presence of addictions.

                        3. **Objects**:
                            - "Total Objects Identified": Integer representing the total number of objects identified.
                            - "Average Features Per Object": Float representing the average number of features per object.
                            - "Objects Details": Dictionary containing details of each object, where each object is labeled as "Object_1", "Object_2", etc., with the following structure:
                                - "Name": The name of the object - as simplest and descriptive mossible.
                                - "Portion Boolean": 0-1 output indicating if the object is a portion of a larger object (1) or a complete object (0). For example, a leg is a portion of a human. However, if the object is just cropped but clearly identifiable as a complete object, it should be considered a complete object.
                                - "Color": The color of the object.
                                - "Features": List of features of the object.
                                - "Total Features": Integer representing the number of features for the object.

                        4. **Place**:
                            - "Name": The name of the place - as simplest and descriptive mossible.
                            - "Certainty Boolean": 0-1 output indicating if the place is clearly identifiable (1) or not (0).
                            - "Fantasy/Adventurous Place": Boolean (0-1) indicating whether the place is classified as a fantasy/adventurous place or not.
                            - "Explanation": Detailed explanation of why the place is classified as fantasy/adventurous or not. Fantasy places are those that do not exist in reality, and adventurous places are defined as those involving clear statements of traveling to space or another country.

                        5. **Characters**:
                            - "Total Characters Identified": Integer representing the total number of characters identified.
                            - "Average Features Per Character": Float representing the average number of features per character.
                            - "Character Details": Dictionary containing details of each character, where each character is labeled as "Character_1", "Character_2", etc., with the following structure:
                                - "Name": The name of the character - as simplest and descriptive mossible.
                                - "Portion Boolean": 0-1 output indicating if the character is a portion of a larger character (1) or a complete character (0). For example, a leg is a portion of a human. However, if the character is just cropped but clearly identifiable as a complete character, it should be considered a complete character.
                                - "Human or Non-Human": 0-1 output indicating if the character appears human (1) or non-human (0). Anthropomorphized characters or any other combination not fully human are considered non-human.
                                - "Physical Features": List of physical features of the character.
                                - "Explanation": Explanation for why the character is classified as human or non-human, and why these physical features are inferred.
                                - "Age": Expected age range of the character (a single number).
                            **Note**: If the "character" consists of only a part of a body (such as a hand, leg, or face without enough distinguishing features to identify it as a complete character), do not count it as a "character."

                        Ensure that the structure of the JSON output strictly adheres to these standardized labels.
                        """
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 750
    }

    for attempt in range(retries):
        try:
            async with aiohttp.ClientSession() as session:
                async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload) as response:
                    # Log the status code and full response for debugging
                    status = response.status
                    response_text = await response.text()
                    
                    # print(f"Response Status Code: {status}")
                    # print(f"Response Content: {response_text}")

                    if status == 429:
                        print("Rate limit exceeded, retrying...")
                        await asyncio.sleep(2 ** attempt)
                        continue
                    elif status == 200:
                        content = await response.json()
                        
                        # Log the full JSON content
                        # print(f"Full JSON Response for {image_path}: {content}")
                        
                        if 'choices' in content:
                            message_content = content['choices'][0].get('message', {}).get('content', '').strip()
                            try:
                                return json.loads(message_content)
                            except json.JSONDecodeError as e:
                                print(f"Error decoding JSON from OpenAI response for {image_path}: {e}")
                                # print(f"OpenAI Response Content: {message_content}")
                                return None
                        else:
                            print(f"Unexpected response format from OpenAI API for {image_path}.")
                            return None
                    else:
                        print(f"Request failed with status code {status} for {image_path}.")
                        # print(f"Response Content: {response_text}")
                        return None
        except aiohttp.ClientError as e:
            print(f"Request failed due to a client error: {e}")
            await asyncio.sleep(2 ** attempt)
        except Exception as e:
            print(f"Unexpected error occurred: {e}")
            await asyncio.sleep(2 ** attempt)
    return None


### 6) Scene Processing Functions

In [20]:
async def process_scenes_output(output_dir, json_output_dir):
    os.makedirs(json_output_dir, exist_ok=True)
    scenes = sorted([f for f in os.listdir(output_dir) if f.endswith('.jpg')], key=extract_scene_number)
    total_scenes = len(scenes)
    with tqdm(total=total_scenes, desc="Processing Scenes", unit="scene") as pbar:
        tasks = [process_single_scene(i, scene, output_dir, json_output_dir, pbar) for i, scene in enumerate(scenes)]
        await asyncio.gather(*tasks)


async def process_single_scene(i, scene, output_dir, json_output_dir, pbar):
    async with semaphore:  # Limit concurrent execution
        scene_path = os.path.join(output_dir, scene)

        # Encode image in base64
        base64_image = await encode_image(scene_path)

        # Perform color analysis
        color_analysis_result = analyze_image_colors(scene_path)

        # Send image to OpenAI for further analysis
        openai_response = await send_image_to_openai(scene_path, base64_image)

        # Check if openai_response is valid (not None or empty)
        if not openai_response:
            print(f"Skipping {scene} due to invalid OpenAI response.")
            pbar.update(1)
            return

        # Combine both results, and include the reference to the image file
        final_output = {
            "Image File": scene,
            "Image Analysis": {
                **color_analysis_result["Color Analysis"],
                **openai_response.get("Image Analysis", {})
            }
        }

        # The filename already includes the scene number and frame number
        output_filename = os.path.splitext(scene)[0] + '_analysis.json'
        output_path = os.path.join(json_output_dir, output_filename)

        try:
            async with aiofiles.open(output_path, 'w') as json_file:
                await json_file.write(json.dumps(final_output, indent=4))
                print(f"Saved analysis for {scene} as {output_filename}")
        except Exception as e:
            print(f"Failed to save analysis for {scene}: {e}")

        pbar.update(1)


def extract_scene_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else -1

def extract_frame_number(filename):
    match = re.search(r'_frame_(\d+)', filename)
    return int(match.group(1)) if match else -1



### 7) Run whole analysis of each json output

Image Path Construction: get_image_path generates the correct path to the image file based on the JSON filename.

Entity Extraction:extract_entities_from_json pulls characters, objects, and places from the JSON data.

Image-to-Image Comparison:perform_image_to_image_comparison compares partial objects with full objects using the OpenAI API.

Entity Comparison:compare_entities handles both name-based and image-based comparisons to decide whether two entities should be consolidated.

Consolidation:Entities across frames are consolidated into a single summary file that tracks where each entity was found.

Main Execution:The script runs through all JSON files, processes the entities, and saves the consolidated results to a summary JSON file.

Key Features of This Implementation:
Text-Based Comparison: The code first attempts to merge entities based on exact name matches. If no match is found, it uses the OpenAI API to determine if two entities with different names should be merged.

Image-to-Image Comparison: If one of the entities is flagged as a portion, or if names don't match but the entities might still be the same, the code performs an image-to-image comparison using the OpenAI API.

Efficient Processing: The code processes each frame sequentially and logs all merges into merged_entities_log, ensuring you have a record of what entities were merged, including their original names and frames.

No Overwritten Functionality: The original image analysis functionality is preserved and integrated smoothly with the text-based comparisons.

Number of Unique Characters, Objects, and Places: This can be done by counting the keys in the consolidated_data dictionary.
Average Characters per Frame: This can be calculated by summing up all instances of characters found across frames and dividing by the total number of frames where characters appear.
Average Features per Character/Object: Calculate this by summing the features of all characters/objects and dividing by the total number of characters/objects.
Overall Color Analysis: Aggregate the color data from all JSON files.
Filter Compliance: Check for any instances where the filters (e.g., nudity, drugs) are not compliant and log the frame numbers.

In [32]:

nest_asyncio.apply()

# Function to construct the image path based on the JSON file
def get_image_path(json_filename, image_directory):
    image_filename = json_filename.replace("_analysis.json", ".jpg")
    return os.path.join(image_directory, image_filename)

# Function to extract entities from a JSON file
def extract_entities_from_json(json_data):
    entities = {
        "characters": [],
        "objects": [],
        "places": []
    }

    if "Image Analysis" in json_data:
        if "Characters" in json_data["Image Analysis"] and json_data["Image Analysis"]["Characters"]["Total Characters Identified"] > 0:
            for character in json_data["Image Analysis"]["Characters"]["Character Details"].values():
                entities["characters"].append(character)
        
        if "Objects" in json_data["Image Analysis"] and json_data["Image Analysis"]["Objects"]["Total Objects Identified"] > 0:
            for obj in json_data["Image Analysis"]["Objects"]["Objects Details"].values():
                entities["objects"].append(obj)
        
        if "Place" in json_data["Image Analysis"]:
            entities["places"].append(json_data["Image Analysis"]["Place"])

    return entities

# Function to encode an image to base64
def encode_image_to_base64(image_path):
    if not image_path or not os.path.isfile(image_path):
        raise ValueError(f"Image path cannot be None or invalid: {image_path}")
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Function to perform image-to-image comparison using OpenAI API
async def perform_image_to_image_comparison(entity1, entity2, image_path1, image_path2, api_key, api_usage):
    base64_image1 = encode_image_to_base64(image_path1)
    base64_image2 = encode_image_to_base64(image_path2)

    prompt = """
    You are an expert in image analysis. Compare the two provided images and determine if they represent the same object or character, even if one is a partial view. Consider features, colors, and context. For instance, if the identified entity is a limb (e.g., a leg), you could contrast that limb with other full views of characters and see if it matches.

    Return 'True' if the images depict the same object or character, 'False' if they are different, and 'Uncertain' if unsure.
    """

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": api_usage.get("model_used", "gpt-4"),  # Fallback to "gpt-4" if not present
        "messages": [{"role": "user", "content": prompt}],
        "images": [{"image": base64_image1}, {"image": base64_image2}],
        "max_tokens": 100
    }

    async with aiohttp.ClientSession() as session:
        try:
            async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload) as response:
                response_json = await response.json()
                
                # Increment API usage counters
                api_usage["total_api_calls"] += 1
                api_usage["total_tokens_used"] += response_json.get("usage", {}).get("total_tokens", 0)
                
                answer = response_json.get('choices', [{}])[0].get('message', {}).get('content', '').strip().lower()
                
                if "uncertain" in answer:
                    return "uncertain"
                elif "true" in answer:
                    return "true"
                else:
                    return "false"
        except KeyError as e:
            print(f"Error accessing API response: {e}")
            return "false"
        except Exception as e:
            print(f"Unexpected error: {e}")
            return "false"




# Function to prompt OpenAI for entity consolidation
async def consolidate_entities_with_openai(entity_type, entity_list, api_key, api_usage, old_to_new_name_map):
    if not entity_list:
        return entity_list  # No entities to process

    # Filter out already consolidated entities
    entity_list = [entity for entity in entity_list if entity not in old_to_new_name_map]

    if not entity_list:
        return []  # Return an empty list if all entities are already consolidated

    # Perform fuzzy consolidation before sending to OpenAI
    consolidated_dict = fuzzy_consolidate_entities(entity_list)
    consolidated_list = list(consolidated_dict.keys())

    # Prepare the prompt for OpenAI API
    prompt = f"""
    You are an expert in entity recognition and consolidation. Here is a list of {entity_type}. The list may include variations in the names or descriptions that refer to the same entity. Please identify which entities refer to the same concept or character and suggest how they should be merged under a single, consistent name. For example, if 'Banana character', 'Banana Character 1', and 'Banana Man' refer to the same entity, suggest that they should be merged under one name.

    List of {entity_type}:
    {', '.join(consolidated_list)}

    Please return a JSON object with the consolidated list of entities where similar entities are merged under a single name.
    """

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 1500  # Increase max_tokens to allow for a more comprehensive response
    }

    async with aiohttp.ClientSession() as session:
        try:
            async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload) as response:
                response_json = await response.json()
                api_usage["total_api_calls"] += 1
                api_usage["total_tokens_used"] += response_json["usage"]["total_tokens"]  # Track the tokens used

                # Log the full JSON response for debugging
                print("OpenAI API Response:", json.dumps(response_json, indent=4))

                consolidated_list = response_json.get('choices', [{}])[0].get('message', {}).get('content', '').strip()
                try:
                    # Update the map with consolidated entities
                    for old_entity in entity_list:
                        old_to_new_name_map[old_entity] = consolidated_list[old_entity]

                    return json.loads(consolidated_list)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from OpenAI response: {e}")
                    return entity_list  # Fallback to original list if parsing fails
        except Exception as e:
            print(f"Error during entity consolidation: {e}")
            return entity_list  # Fallback to original list if API call fails



# Match words with FuzzyWuzzy
def fuzzy_consolidate_entities(entity_list, threshold=80):
    consolidated_entities = {}
    for entity in entity_list:
        if consolidated_entities:
            result = process.extractOne(entity, consolidated_entities.keys(), scorer=fuzz.ratio)
            if result:
                best_match, score = result[0], result[1]
                if score is not None and score >= threshold:
                    consolidated_entities[best_match].append(entity)
                else:
                    consolidated_entities[entity] = [entity]
            else:
                consolidated_entities[entity] = [entity]
        else:
            consolidated_entities[entity] = [entity]
    return consolidated_entities



# Function to apply the consolidation based on the OpenAI response
def apply_consolidation(consolidated_data, entity_type, new_entity_list, merged_entities_log):
    old_to_new_map = {}
    for new_entity in new_entity_list:
        if isinstance(new_entity, dict):
            for old_entity in new_entity.get("merge", []):  # Safely get "merge" list
                old_to_new_map[old_entity] = new_entity["name"]

    for old_entity, new_entity in old_to_new_map.items():
        if old_entity in consolidated_data[entity_type]:
            if new_entity not in consolidated_data[entity_type]:
                consolidated_data[entity_type][new_entity] = consolidated_data[entity_type][old_entity]
            else:
                consolidated_data[entity_type][new_entity]["frames_found"].extend(consolidated_data[entity_type][old_entity]["frames_found"])
                consolidated_data[entity_type][new_entity]["frames_found"] = list(set(consolidated_data[entity_type][new_entity]["frames_found"]))

            merged_entities_log.append({
                "merged_to": new_entity,
                "merged_from": old_entity,
                "frames": consolidated_data[entity_type][old_entity]["frames_found"]
            })
            del consolidated_data[entity_type][old_entity]

    return consolidated_data, merged_entities_log


# Function to generate summary statistics
def generate_summary_statistics(consolidated_data, all_json_files_data):
    summary_statistics = {
        "Number of Unique Characters": len(consolidated_data["characters"]),
        "Number of Unique Objects": len(consolidated_data["objects"]),
        "Number of Unique Places": len(consolidated_data["places"]),
    }

    # Initialize variables for aggregation
    total_characters_identified = 0
    total_objects_identified = 0
    total_frames = len(all_json_files_data)
    total_character_features = 0
    total_object_features = 0
    color_totals = {
        "Red": 0,
        "Yellow": 0,
        "Green": 0,
        "Blue": 0,
        "White": 0,
        "Black": 0,
        "Non-primary": 0
    }
    non_compliant_frames = []
    fantasy_adventurous_count = 0

    # Aggregate data from all JSON files
    for json_data in all_json_files_data:
        image_analysis = json_data["Image Analysis"]

        # Summing up total identified characters and objects
        total_characters_identified += image_analysis["Characters"]["Total Characters Identified"]
        total_objects_identified += image_analysis["Objects"]["Total Objects Identified"]

        # Summing up features per character and object with a default value of 0 if "Total Features" is missing
        total_character_features += sum([char.get("Total Features", 0) for char in image_analysis["Characters"]["Character Details"].values()])
        total_object_features += sum([obj.get("Total Features", 0) for obj in image_analysis["Objects"]["Objects Details"].values()])

        # Aggregating color data
        for color, color_data in image_analysis["Colors Found"].items():
            color_totals[color] += color_data["Pixel Count"]

        # Check compliance and log non-compliant frames
        for filter_name, is_compliant in image_analysis["Suitability"].items():
            if is_compliant:  # Log only if true, indicating non-compliance
                non_compliant_frames.append({
                    "Frame": json_data["Image File"],
                    "Non-Compliant Filter": filter_name
                })

        # Count fantasy/adventurous places
        if image_analysis["Place"].get("Fantasy/Adventurous Place", 0) == 1:
            fantasy_adventurous_count += 1

    # Calculate averages
    summary_statistics["Average Characters per Frame"] = total_characters_identified / total_frames if total_frames > 0 else 0
    summary_statistics["Average Features per Character"] = total_character_features / total_characters_identified if total_characters_identified > 0 else 0
    summary_statistics["Average Features per Object"] = total_object_features / total_objects_identified if total_objects_identified > 0 else 0

    # Add overall color percentages
    total_pixels = sum(color_totals.values())
    color_percentages = {color: (count / total_pixels) * 100 for color, count in color_totals.items() if total_pixels > 0}
    summary_statistics["Overall Color Distribution"] = color_percentages

    # Add compliance information
    summary_statistics["Non-Compliant Frames"] = non_compliant_frames

    # Calculate and add the percentage of fantasy/adventurous places
    summary_statistics["Percentage of Fantasy/Adventurous Places"] = (fantasy_adventurous_count / total_frames) * 100 if total_frames > 0 else 0

    return summary_statistics


# Function to print the consolidated entities
def print_consolidated_data(consolidated_data, title="Consolidated"):
    print(f"\n{title} Characters:")
    for char_name, char_data in consolidated_data["characters"].items():
        print(f"{char_name}: Found in frames {char_data['frames_found']}")

    print(f"\n{title} Objects:")
    for obj_name, obj_data in consolidated_data["objects"].items():
        print(f"{obj_name}: Found in frames {obj_data['frames_found']}")

    print(f"\n{title} Places:")
    for place_name, place_data in consolidated_data["places"].items():
        print(f"{place_name}: Found in frames {place_data['frames_found']}")

# Function to initially extract and consolidate all entities into lists
async def initial_consolidation(json_output_dir, image_output_dir):
    consolidated_data = {
        "characters": {},
        "objects": {},
        "places": {}
    }

    json_files = [f for f in os.listdir(json_output_dir) if f.endswith('.json')]
    
    for json_file in json_files:
        json_path = os.path.join(json_output_dir, json_file)
        with open(json_path, 'r') as file:
            json_data = json.load(file)

        frame_number = extract_frame_number(json_file)
        entities = extract_entities_from_json(json_data)

        for entity_type in ["characters", "objects", "places"]:
            for new_entity in entities[entity_type]:
                entity_name = new_entity["Name"]
                
                # Generate the image path
                image_path = get_image_path(json_file, image_output_dir)
                new_entity["Image Path"] = image_path
                
                if entity_name in consolidated_data[entity_type]:
                    consolidated_data[entity_type][entity_name]["frames_found"].append(frame_number)
                else:
                    consolidated_data[entity_type][entity_name] = {
                        "entity": new_entity,
                        "frames_found": [frame_number]
                    }

    return consolidated_data

# Function to process and consolidate all entities
async def consolidate_all_entities(consolidated_data, api_key, api_usage, old_to_new_name_map):
    merged_entities_log = []

    # Consolidate characters
    characters = list(consolidated_data["characters"].keys())
    fuzzy_consolidated_characters = fuzzy_consolidate_entities(characters, threshold=80)  # Apply fuzzy matching here
    new_characters = await consolidate_entities_with_openai("characters", fuzzy_consolidated_characters, api_key, api_usage, old_to_new_name_map)
    consolidated_data, merged_entities_log = apply_consolidation(consolidated_data, "characters", new_characters, merged_entities_log)

    # Consolidate objects
    objects = list(consolidated_data["objects"].keys())
    fuzzy_consolidated_objects = fuzzy_consolidate_entities(objects, threshold=80)  # Apply fuzzy matching here
    new_objects = await consolidate_entities_with_openai("objects", fuzzy_consolidated_objects, api_key, api_usage, old_to_new_name_map)
    consolidated_data, merged_entities_log = apply_consolidation(consolidated_data, "objects", new_objects, merged_entities_log)

    # Consolidate places
    places = list(consolidated_data["places"].keys())
    fuzzy_consolidated_places = fuzzy_consolidate_entities(places, threshold=80)  # Apply fuzzy matching here
    new_places = await consolidate_entities_with_openai("places", fuzzy_consolidated_places, api_key, api_usage, old_to_new_name_map)
    consolidated_data, merged_entities_log = apply_consolidation(consolidated_data, "places", new_places, merged_entities_log)

    return consolidated_data, merged_entities_log




# Function to handle the image-to-image comparisons for portions
async def handle_portion_comparisons(consolidated_data, api_key, api_usage):
    merged_entities_log = []

    for entity_type in ["characters", "objects", "places"]:
        for entity_name, entity_data in consolidated_data[entity_type].items():
            if entity_data["entity"].get("Portion Boolean") == 1:
                # Perform image-to-image comparisons for portions
                for other_entity_name, other_entity_data in consolidated_data[entity_type].items():
                    if other_entity_name != entity_name and other_entity_data["entity"].get("Portion Boolean") != 1:
                        comparison_result = await perform_image_to_image_comparison(
                            entity_data["entity"], other_entity_data["entity"],
                            entity_data["entity"]["Image Path"], other_entity_data["entity"]["Image Path"],
                            api_key, api_usage  # Pass the api_usage dictionary here
                        )
                        if comparison_result == "true":
                            if entity_name not in other_entity_data["frames_found"]:
                                other_entity_data["frames_found"].extend(entity_data["frames_found"])
                                other_entity_data["frames_found"] = list(set(other_entity_data["frames_found"]))

                            merged_entities_log.append({
                                "merged_to": other_entity_name,
                                "merged_from": entity_name,
                                "frames": entity_data["frames_found"]
                            })
                            del consolidated_data[entity_type][entity_name]
                            break

    return consolidated_data, merged_entities_log


# Function to perform a final image-to-image comparison for characters
async def final_image_to_image_comparison_for_characters(consolidated_data, api_key, api_usage):
    character_names = list(consolidated_data["characters"].keys())
    merged_entities_log = []

    # Progress bar for character comparisons
    with tqdm(total=len(character_names) * (len(character_names) - 1) // 2, desc="Final Character Comparisons", unit="comparison") as pbar:
        # Compare each character with every other character
        for i in range(len(character_names)):
            for j in range(i + 1, len(character_names)):
                name1 = character_names[i]
                name2 = character_names[j]

                entity1 = consolidated_data["characters"][name1]["entity"]
                entity2 = consolidated_data["characters"][name2]["entity"]

                # Perform image comparison
                comparison_result = await perform_image_to_image_comparison(
                    entity1, entity2,
                    entity1["Image Path"], entity2["Image Path"],
                    api_key, api_usage  # Pass the api_usage dictionary here
                )

                if comparison_result == "true":
                    # Merge name2 into name1
                    consolidated_data["characters"][name1]["frames_found"].extend(consolidated_data["characters"][name2]["frames_found"])
                    consolidated_data["characters"][name1]["frames_found"] = list(set(consolidated_data["characters"][name1]["frames_found"]))

                    # Log the merge
                    merged_entities_log.append({
                        "merged_to": name1,
                        "merged_from": name2,
                        "frames": consolidated_data["characters"][name2]["frames_found"]
                    })

                    # Remove the merged entity
                    del consolidated_data["characters"][name2]

                pbar.update(1)  # Update the progress bar

    return consolidated_data, merged_entities_log


# Function to save the summary to a JSON file
def save_summary_to_json(consolidated_data, output_path, video_title, file_size, processing_time, merged_entities_log, all_json_files_data, api_usage, scenes_per_minute, number_of_scenes, video_length):
    # Generate additional summary statistics
    summary_statistics = generate_summary_statistics(consolidated_data, all_json_files_data)

    summary = {
        "Video Title": video_title,
        "File Size (bytes)": file_size,
        "Video Length (seconds)": video_length,
        "Number of Scenes": number_of_scenes,
        "Scenes per Minute": scenes_per_minute,
        "Processing Time (seconds)": processing_time,
        "API Usage": api_usage,
        "Summary Statistics": summary_statistics,
        "Consolidated Data": consolidated_data,
        "Merged Entities Log": merged_entities_log
    }
    
    with open(output_path, 'w') as f:
        json.dump(summary, f, indent=4)


# Extract frame number from file name
def extract_frame_number(filename):
    match = re.search(r'_frame_(\d+)', filename)
    return int(match.group(1)) if match else None




In [33]:
# ############################
# # TEXT FOR ANALYSIS ,... FEEL FREE TO COMMENT

# import os
# import json
# import nest_asyncio
# import asyncio

# # Set your directories and variables
# json_output_dir = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Bananas_in_pyjamas/json_output'
# image_output_dir = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Bananas_in_pyjamas/scenes_output'
# summary_json_path = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Bananas_in_pyjamas_summary.json'
# video_title = "Bananas_in_pyjamas.mp4"
# api_key = os.getenv("OPENAI_API_KEY")

# # Apply nest_asyncio to handle the running event loop
# nest_asyncio.apply()

# async def perform_image_to_image_comparison(entity1, entity2, image_path1, image_path2, api_key, api_usage):
#     base64_image1 = encode_image_to_base64(image_path1)
#     base64_image2 = encode_image_to_base64(image_path2)

#     prompt = """
#     You are an expert in image analysis. Compare the two provided images and determine if they represent the same object or character, even if one is a partial view. Consider features, colors, and context. For instance, if the identified entity is a limb (e.g., a leg), you could contrast that limb with other full views of characters and see if it matches.

#     Return 'True' if the images depict the same object or character, 'False' if they are different, and 'Uncertain' if unsure.
#     """

#     headers = {
#         "Content-Type": "application/json",
#         "Authorization": f"Bearer {api_key}"
#     }

#     payload = {
#         "model": api_usage.get("model_used", "gpt-4"),  # Fallback to "gpt-4" if not present
#         "messages": [{"role": "user", "content": prompt}],
#         "images": [{"image": base64_image1}, {"image": base64_image2}],
#         "max_tokens": 100
#     }

#     async with aiohttp.ClientSession() as session:
#         try:
#             async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload) as response:
#                 response_json = await response.json()
                
#                 # Increment API usage counters
#                 api_usage["total_api_calls"] += 1
#                 api_usage["total_tokens_used"] += response_json.get("usage", {}).get("total_tokens", 0)
                
#                 answer = response_json.get('choices', [{}])[0].get('message', {}).get('content', '').strip().lower()
                
#                 if "uncertain" in answer:
#                     return "uncertain"
#                 elif "true" in answer:
#                     return "true"
#                 else:
#                     return "false"
#         except KeyError as e:
#             print(f"Error accessing API response: {e}")
#             return "false"
#         except Exception as e:
#             print(f"Unexpected error: {e}")
#             return "false"


# # Run the test
# asyncio.get_event_loop().run_until_complete(test_entity_consolidation())


### 8) Main Function Execution

In [34]:
def process_videos_in_directory(directory_path, output_base_dir):
    video_files = [f for f in os.listdir(directory_path) if f.endswith(('.mp4', '.avi', '.mkv'))]

    if not video_files:
        print("No video files found in the directory.", flush=True)
        return

    with tqdm(total=len(video_files), desc="Processing Videos", unit="video") as pbar:
        for i, video_file in enumerate(video_files):
            start_time = time.time()

            try:
                video_path = os.path.join(directory_path, video_file)
                video_name = os.path.splitext(video_file)[0]
                video_size = os.path.getsize(video_path)

                video_output_dir = os.path.join(output_base_dir, video_name)
                scenes_output_dir = os.path.join(video_output_dir, 'scenes_output')
                json_output_dir = os.path.join(video_output_dir, 'json_output')

                os.makedirs(scenes_output_dir, exist_ok=True)

                print(f"Processing video {i + 1}/{len(video_files)}: {video_file}", flush=True)

                scenes = analyze_video(video_path)
                extract_frames_imageio(video_path, scenes, scenes_output_dir)
                asyncio.run(process_scenes_output(scenes_output_dir, json_output_dir))  # Run async scene processing

                end_time = time.time()
                processing_time = end_time - start_time

                # Calculate video length (in seconds)
                video_length = get_video_length(video_path)

                # Calculate scenes per minute
                scenes_per_minute = len(scenes) / (video_length / 60)

                # Additional processing (entity consolidation and comparison)
                asyncio.run(run_additional_processing(json_output_dir, scenes_output_dir, video_file, video_size, processing_time, scenes_per_minute, len(scenes), video_length))

                print(f"Finished processing video: {video_file}", flush=True)
                pbar.update(1)
            except Exception as e:
                print(f"Error processing video {video_file}: {e}", flush=True)

async def run_additional_processing(json_output_dir, image_output_dir, video_file, video_size, processing_time, scenes_per_minute, number_of_scenes, video_length):
    api_key = os.getenv("OPENAI_API_KEY")

    # Initialize API usage tracking
    api_usage = {
        "total_api_calls": 0,
        "total_tokens_used": 0,
        "model_used": "gpt-4"  # Assuming you're using GPT-4
    }

    old_to_new_name_map = {}  # Initialize the mapping dictionary

    start_time = time.time()

    # Load all JSON files data
    all_json_files_data = []
    json_files = [f for f in os.listdir(json_output_dir) if f.endswith('.json')]
    for json_file in json_files:
        with open(os.path.join(json_output_dir, json_file), 'r') as file:
            all_json_files_data.append(json.load(file))

    with tqdm(total=4, desc="Overall Progress", unit="step") as overall_pbar:
        try:
            # Step 1: Initial extraction and consolidation
            consolidated_data = await initial_consolidation(json_output_dir, image_output_dir)
            print_consolidated_data(consolidated_data, title="Initial")
            overall_pbar.update(1)

            # Step 2: Consolidate entities using OpenAI
            consolidated_data, merged_entities_log = await consolidate_all_entities(consolidated_data, api_key, api_usage, old_to_new_name_map)
            print_consolidated_data(consolidated_data, title="After OpenAI Consolidation")
            overall_pbar.update(1)

            # Step 3: Handle image-to-image comparisons for portions
            consolidated_data, portion_merged_log = await handle_portion_comparisons(consolidated_data, api_key, api_usage)
            merged_entities_log.extend(portion_merged_log)
            print_consolidated_data(consolidated_data, title="After Portion Comparisons")
            overall_pbar.update(1)

            # Step 4: Final image-to-image comparison for characters
            consolidated_data, final_character_merged_log = await final_image_to_image_comparison_for_characters(consolidated_data, api_key, api_usage)
            merged_entities_log.extend(final_character_merged_log)
            print_consolidated_data(consolidated_data, title="Final After Image Comparisons")
            overall_pbar.update(1)

        except Exception as e:
            print(f"Error during additional processing for video {video_file}: {e}", flush=True)

    end_time = time.time()
    processing_time += (end_time - start_time)

    # Save the final summary using the comprehensive `save_summary_to_json` function
    summary_output_path = os.path.join(os.path.dirname(json_output_dir), f"{os.path.basename(json_output_dir)}_summary.json")
    save_summary_to_json(consolidated_data, summary_output_path, video_file, video_size, processing_time, merged_entities_log, all_json_files_data, api_usage, scenes_per_minute, number_of_scenes, video_length)

# Main script execution
old_to_new_name_map = {}
video_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/02_Video_DB'
output_base_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB'

process_videos_in_directory(video_directory, output_base_directory)
print("FINISHED PROCESSING ALL VIDEOS.", flush=True)

Processing Videos:   0%|          | 0/1 [00:00<?, ?video/s]

Processing video 1/1: Bananas_in_pyjamas.mp4


ERROR:pyscenedetect:VideoManager is deprecated and will be removed.
INFO:pyscenedetect:Loaded 1 video, framerate: 25.000 FPS, resolution: 640 x 360
INFO:pyscenedetect:Downscale factor set to 2, effective resolution: 320 x 180
INFO:pyscenedetect:Detecting scenes...
INFO:root:Detected 159 scenes:
INFO:root:Scene 1: Start 00:00:00.000 / Frame 0, End 00:00:08.360 / Frame 209
INFO:root:Scene 2: Start 00:00:08.360 / Frame 209, End 00:00:19.320 / Frame 483
INFO:root:Scene 3: Start 00:00:19.320 / Frame 483, End 00:00:24.040 / Frame 601
INFO:root:Scene 4: Start 00:00:24.040 / Frame 601, End 00:00:24.800 / Frame 620
INFO:root:Scene 5: Start 00:00:24.800 / Frame 620, End 00:00:26.600 / Frame 665
INFO:root:Scene 6: Start 00:00:26.600 / Frame 665, End 00:00:28.600 / Frame 715
INFO:root:Scene 7: Start 00:00:28.600 / Frame 715, End 00:00:29.960 / Frame 749
INFO:root:Scene 8: Start 00:00:29.960 / Frame 749, End 00:00:31.880 / Frame 797
INFO:root:Scene 9: Start 00:00:31.880 / Frame 797, End 00:00:35.24

Extracted and saved middle frame of scene 1 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Bananas_in_pyjamas/scenes_output/scene_1_frame_104.jpg
Extracted and saved middle frame of scene 2 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Bananas_in_pyjamas/scenes_output/scene_2_frame_346.jpg
Extracted and saved middle frame of scene 3 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Bananas_in_pyjamas/scenes_output/scene_3_frame_542.jpg
Extracted and saved middle frame of scene 4 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Bananas_in_pyjamas/scenes_output/scene_4_frame_610.jpg
Extracted and saved middle frame of scene 5 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Bananas_in_pyjamas/scenes_output/scene_5_frame_642.jpg
Extracted and saved middle frame of scene 6 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesi

Saved analysis for scene_4_frame_610.jpg as scene_4_frame_610_analysis.json


...

OpenAI API Response: {
    "id": "chatcmpl-A3RbKgClt4sVNkygtPBocFQvOT9mK",
    "object": "chat.completion",
    "created": 1725384446,
    "model": "gpt-4-0613",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "{\n    \"Cartoon Environment\": [\"Cartoon Landscape\", \"Cartoon environment\", \"Outdoor Scene\", \"Cartoon World\", \"Cartoon Setting\", \"Cartoon Garden\", \"Cartoon Background\", \"Cartoon Outdoor Scene\", \"Cartoon outdoor setting\", \"Animated Environment\", \"Cartoon Forest Scene\", \"Cartoon Wilderness\"],\n    \"Outdoor Area\": [\"Outdoor\", \"Outdoor Park\", \"Outdoor Area\", \"Outdoor Backyard\",\"Outdoor grass setting\",\"Outdoor Setting\"],\n    \"Park\": [\"Park\", \"Garden\", \"Garden Path\", \"Outdoor Park\", \"Cartoon Garden\", \"Cartoon backyard\", \"Backyard\"],\n    \"Simple Background\": [\"Sky\", \"Blue background\", \"Simple Background\", \"Abstract Background\"],\


After Portion Comparisons Characters:
Banana character: Found in frames [2622, 14314, 14314, 2995, 8673, 8673, 14733, 2286, 3126, 2129, 12998, 12998]
Brown bear character 1: Found in frames [6862]
Brown bear character 2: Found in frames [6862]
Yellow bear character 3: Found in frames [6862]
Banana 1: Found in frames [12768, 10438, 4430, 346, 3780, 15473, 5101, 12382, 7715, 773, 1907]
Banana 2: Found in frames [12768, 10438, 4430, 346, 3780, 15473, 5101, 12382, 7715, 773, 1907]
Mouse: Found in frames [12768, 11811, 9529, 9391, 4098, 12113, 15473, 2000, 5101, 15187, 14473, 1162, 1411, 4503, 1907]
Banana Character 1: Found in frames [8745, 6271, 4707, 16482, 14551, 1812, 8136, 13397, 839, 8821, 3342, 16616, 8497, 6014, 16324, 13240, 3548, 5460, 4559, 13612, 542, 8952, 15808]
Banana Character 2: Found in frames [8745, 6271, 4707, 16482, 14551, 1812, 8136, 13397, 839, 8821, 3342, 16616, 8497, 6014, 16324, 13240, 3548, 5460, 4559, 13612, 542, 8952, 15808]
Cartoon Mouse: Found in frames [422


Final After Image Comparisons Characters:
Banana character: Found in frames [2622, 14314, 14314, 2995, 8673, 8673, 14733, 2286, 3126, 2129, 12998, 12998]
Brown bear character 1: Found in frames [6862]
Brown bear character 2: Found in frames [6862]
Yellow bear character 3: Found in frames [6862]
Banana 1: Found in frames [12768, 10438, 4430, 346, 3780, 15473, 5101, 12382, 7715, 773, 1907]
Banana 2: Found in frames [12768, 10438, 4430, 346, 3780, 15473, 5101, 12382, 7715, 773, 1907]
Mouse: Found in frames [12768, 11811, 9529, 9391, 4098, 12113, 15473, 2000, 5101, 15187, 14473, 1162, 1411, 4503, 1907]
Banana Character 1: Found in frames [8745, 6271, 4707, 16482, 14551, 1812, 8136, 13397, 839, 8821, 3342, 16616, 8497, 6014, 16324, 13240, 3548, 5460, 4559, 13612, 542, 8952, 15808]
Banana Character 2: Found in frames [8745, 6271, 4707, 16482, 14551, 1812, 8136, 13397, 839, 8821, 3342, 16616, 8497, 6014, 16324, 13240, 3548, 5460, 4559, 13612, 542, 8952, 15808]
Cartoon Mouse: Found in frames 


Processing Videos: 100%|██████████| 1/1 [34:30<00:00, 2070.65s/video]

FINISHED PROCESSING ALL VIDEOS.
